In [12]:
import os
import cv2
import time
import torch
import torchvision
import numpy as np
import torch.nn as nn
device = torch.device("cpu")

model_path = 'best.pt'
updated_class_labels = ['out_of_focus_cervix', 'in_focus_cervix', 'non_cervix']
input_file_name = r'A12 AVE 030633 B.jpg'

#nms_pred = non_max_suppression(pred)#, opt.conf_thres, opt.iou_thres, classes=opt.classes, agnostic=opt.agnostic_nms)
#print(nms_pred)

In [13]:
# Define supporting functions
def autopad(k, p=None):  # kernel, padding
    # Pad to 'same'
    if p is None:
        p = k // 2 if isinstance(k, int) else [x // 2 for x in k]  # auto-pad
    return p

class Conv(nn.Module):
    # Standard convolution
    def __init__(self, c1, c2, k=1, s=1, p=None, g=1, act=True):  # ch_in, ch_out, kernel, stride, padding, groups
        super(Conv, self).__init__()
        self.conv = nn.Conv2d(c1, c2, k, s, autopad(k, p), groups=g, bias=False)
        self.bn = nn.BatchNorm2d(c2)
        self.act = nn.SiLU() if act is True else (act if isinstance(act, nn.Module) else nn.Identity())

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

    def fuseforward(self, x):
        return self.act(self.conv(x))

class Ensemble(nn.ModuleList):
    # Ensemble of models
    def __init__(self):
        super(Ensemble, self).__init__()

    def forward(self, x, augment=False):
        y = []
        for module in self:
            y.append(module(x, augment)[0])
        # y = torch.stack(y).max(0)[0]  # max ensemble
        # y = torch.stack(y).mean(0)  # mean ensemble
        y = torch.cat(y, 1)  # nms ensemble
        return y, None  # inference, train output
    
def attempt_load(weights, map_location=None):
    # Loads an ensemble of models weights=[a,b,c] or a single model weights=[a] or weights=a
    model = Ensemble()
    for w in weights if isinstance(weights, list) else [weights]:
        ckpt = torch.load(w, map_location=map_location)  # load
        model.append(ckpt['ema' if ckpt.get('ema') else 'model'].float().fuse().eval())  # FP32 model
    
    # Compatibility updates
    for m in model.modules():
        if type(m) in [nn.Hardswish, nn.LeakyReLU, nn.ReLU, nn.ReLU6, nn.SiLU]:
            m.inplace = True  # pytorch 1.7.0 compatibility
        elif type(m) is nn.Upsample:
            m.recompute_scale_factor = None  # torch 1.11.0 compatibility
        elif type(m) is Conv:
            m._non_persistent_buffers_set = set()  # pytorch 1.6.0 compatibility
    
    if len(model) == 1:
        return model[-1]  # return model
    else:
        print('Ensemble created with %s\n' % weights)
        for k in ['names', 'stride']:
            setattr(model, k, getattr(model[-1], k))
        return model  # return ensemble


In [14]:
image_data = cv2.imread(input_file_name) 
image_data = image_data[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x41
image_data = np.expand_dims(image_data, axis=0) 
image_data = np.ascontiguousarray(image_data)

# Convert images from numpy array to torch tensor.
image_data = torch.from_numpy(image_data)
image_data = image_data.float()  # uint8 to fp16/32
image_data /= 255.0  # 0 - 255 to 0.0 - 1.0

image_data = image_data.to(device)

model = attempt_load(model_path, map_location=torch.device('cpu'))
model.to(device)
pred = model(image_data)
#print(len(pred[0])) # pred[0] : Tensor of size [1, 9072, 7]
# print(len(pred[1])) # len = 3 
print(pred[1][2][0].shape) # pred[1] is a list with 3 item each of length 5
# print(pred[1][1][0].shape) = torch.Size([3, 24, 24, 7])
# print(pred[1][0][0].shape) = torch.Size([3, 48, 48, 7])
# print(pred[1][2][0].shape) = torch.Size([3, 12, 12, 7])



Fusing layers... 
IDetect.fuse
torch.Size([3, 12, 12, 7])


In [15]:
# How to inpretate number of pred 
image_predictions = pred[0][0] # first zero is to read inference, second zero is to read image idx (first image out of current batch)

boxes_confs = image_predictions[:,5]
cls_confs = image_predictions[:,4]

for box_idx, box in enumerate(image_predictions):
    center_x, center_y, width, height = box[:4]
    box_conf = boxes_confs[box_idx]
    
    print(box_conf, center_x, center_y, width, height)
    

tensor(0.41631) tensor(6.34607) tensor(6.02879) tensor(20.50190) tensor(20.13464)
tensor(0.37838) tensor(13.02146) tensor(6.23424) tensor(24.01483) tensor(14.87469)
tensor(0.36917) tensor(21.22984) tensor(6.16710) tensor(24.53289) tensor(14.13465)
tensor(0.38126) tensor(28.82185) tensor(6.08020) tensor(19.94453) tensor(17.96809)
tensor(0.54130) tensor(36.85984) tensor(5.96468) tensor(27.27438) tensor(15.65640)
tensor(0.61063) tensor(44.60275) tensor(5.92363) tensor(26.92276) tensor(17.49420)
tensor(0.63167) tensor(52.46322) tensor(5.96824) tensor(28.03310) tensor(16.81266)
tensor(0.64075) tensor(60.46761) tensor(6.01957) tensor(27.94909) tensor(17.37756)
tensor(0.62192) tensor(68.16606) tensor(5.90078) tensor(27.77542) tensor(16.78292)
tensor(0.60339) tensor(76.24606) tensor(5.73577) tensor(28.00811) tensor(16.30716)
tensor(0.58126) tensor(84.31339) tensor(5.74728) tensor(27.32624) tensor(16.45338)
tensor(0.57055) tensor(92.06863) tensor(5.90765) tensor(25.91862) tensor(17.03415)
tenso